In [1]:
from typing import Any

import lightning.pytorch as pl
import torch
from lightning.pytorch.callbacks import ModelCheckpoint
import mlflow
from torch.utils.data import DataLoader, Dataset

from src.ml.data.splitting import create_data_splits
from src.ml.modeling import (
    model_factory,
    optimizer_factory,
)
from src.ml.utils.set_seed import set_seed

from datetime import datetime
from loguru import logger
from pathlib import Path
from shutil import copy, copytree, rmtree

import yaml

from src.ml.train_neural_network import train_neural_network
from src.ml.data import data_sets_factory
from src.ml.preprocessing import preprocessing_factory
from src.ml.utils.set_seed import set_seed

In [2]:
CONFIG_FILE = Path("src/ml/config.yaml")

In [3]:
torch.set_default_device(torch.device("cpu"))

In [4]:
# load config file

logger.info("Loading config file.")

with open(CONFIG_FILE, "r") as f:
    config = yaml.safe_load(f)

2024-12-13 02:40:39.304 | INFO     | __main__:<module>:3 - Loading config file.


In [5]:
# load data

logger.info("Loading data.")

data_sets = data_sets_factory(**config["data_set"])
logger.info("Loaded {} data sets.", len(data_sets))

# preprocess data

logger.info("Start preprocessing.")

for preprocessing_step in config["preprocessing"]:
    logger.info("Perform {} preprocessing.", preprocessing_step["name"])

    transform = preprocessing_factory(**preprocessing_step)
    data_sets = [transform(data_set) for data_set in data_sets]

dataset = data_sets[0]

2024-12-13 02:40:39.309 | INFO     | __main__:<module>:3 - Loading data.
2024-12-13 02:40:39.310 | INFO     | __main__:<module>:6 - Loaded 1 data sets.
2024-12-13 02:40:39.311 | INFO     | __main__:<module>:10 - Start preprocessing.


In [6]:
train_dataset, val_dataset, test_dataset = create_data_splits(
    dataset, **config["training"]["splitting_config"]
)

In [7]:
train_loader = DataLoader(train_dataset, **config["training"]["dataloader_config"])
test_loader = DataLoader(test_dataset,batch_size=1)
val_loader = DataLoader(val_dataset)

optimizer = optimizer_factory(**config["training"]["optimizer_config"])
model = model_factory(
    optimizer=optimizer,
    dim=len(train_dataset[0]["branch_lengths"]),
    **config["training"]["model_config"],
)

In [35]:
from src.ml.modeling.conditional_tree_flow import ConditionalTreeFlow

model = ConditionalTreeFlow.load_from_checkpoint("ml_data/models/debug_simple_conditional_flow/epoch=95-val_loss=-8469.78.ckpt")

In [36]:
sample = next(iter(test_loader))
sample

{'branch_lengths': tensor([[-0.0002,  0.0018,  0.0031,  0.0020,  0.0019,  0.0007,  0.0027, -0.0002,
          -0.0004, -0.0010]]),
 'clades_one_hot': tensor([[0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0.,
          1., 0.]])}

In [37]:
latent = model.forward(sample)
latent

{'z': tensor([[-0.2234,  0.7511,  2.4736,  1.3642,  1.1925,  1.3798,  1.2908,  0.0146,
           0.5331, -2.2469]], grad_fn=<MulBackward0>),
 'context': tensor([[0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0.,
          1., 0.]]),
 'log_dj': tensor([49.7610], grad_fn=<AddBackward0>)}

In [38]:
output = model.inverse(latent)
output

{'branch_lengths': tensor([[-0.0002,  0.0018,  0.0031,  0.0020,  0.0019,  0.0007,  0.0027, -0.0002,
          -0.0004, -0.0010]], grad_fn=<SubBackward0>),
 'clades_one_hot': tensor([[0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0.,
          1., 0.]]),
 'log_dj': tensor([49.7610], grad_fn=<AddBackward0>)}

In [44]:
latent["z"] = model.prior.sample((10,))
output = model.inverse(latent)
latent["z"], output

(tensor([-0.0208,  0.2123,  0.0353, -0.1733, -0.0747,  0.0670, -0.0760, -0.0819,
         -0.0205, -0.0662], grad_fn=<SubBackward0>),
 {'branch_lengths': tensor([-0.0208,  0.2123,  0.0353, -0.1733, -0.0747,  0.0670, -0.0760, -0.0819,
          -0.0205, -0.0662], grad_fn=<SubBackward0>),
  'clades_one_hot': tensor([[0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0.,
           1., 0.]]),
  'log_dj': tensor([49.7610], grad_fn=<AddBackward0>)})